<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2d2e66944858397e71b9b2141ee4890bb25a489753f0f95650f1e16e884853e1
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-20 12:29:18
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.37 C
-------------------
Today PnL: -1.98 L (-1.43%)
Current PnL: -31.91 L (-20.35%)
CY Booked + Current PnL: -17.62 L (-11.24%)
-------------------
Total profit:  1.38 L
Total loss:  -33.29 L
-------------------
Total Booked + Current PnL: 9.36 L (7.21%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.42%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 97.27 L (70.91%)
Deployed:  1.30 C
Current:  1.37 C
CAGR/XIRR %: 2.93%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.52,15.38,5.22,21.40,15986.0,40810.0,306236.0,7.09,59.0,X-LC,6.93,14.0,2.55,2.23,22.68,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,2.25,-22.88,29.79,0.09,23152.0,-23061.0,77716.0,80.36,39.0,M-SC,4.69,253.0,-1.00,0.57,2.25,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-2.46,3.17,15.56,19.23,24154.0,4774.0,155234.0,21.35,55.0,L-LC,10.41,182.0,0.20,1.13,25.12,XY25,ATH,MINING
49,MASFIN,398.61,-0.98,-5.85,29.63,22.05,27334.0,-5730.0,92250.0,-18.70,43.0,H-SC,5.01,168.0,-0.21,0.67,34.09,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.96,-19.14,23.69,0.02,33438.0,-33410.0,141149.0,115.40,40.0,M-SC,9.09,240.0,-1.00,1.03,22.78,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,2.25,-22.88,29.79,0.09,23152.0,-23061.0,77716.0,80.36,39.0,M-SC,4.69,253.0,-1.00,0.57,2.25,OX40N,NTT,DURABLES
33,HINDUNILVR,2922.00,0.58,-5.56,20.36,13.67,49430.0,-14283.0,242780.0,-12.61,62.0,X-LC,2.34,9.0,-0.29,1.77,14.61,XY25,NTT,FMCG
34,HONAUT,58357.33,0.53,-16.74,72.15,43.32,97835.0,-27272.0,135600.0,-28.43,51.0,X-SC,8.11,90.0,-0.28,0.99,4.12,X40N,ATH,ELECTRICAL
14,CAMPUS,393.00,0.14,-15.50,57.74,33.28,81856.0,-26009.0,141766.0,-31.99,32.0,M-SC,4.57,221.0,-0.32,1.03,9.88,XY24,NTT,FOOTWEAR
68,STARHEALTH,761.00,0.09,-15.08,73.41,47.26,175255.0,-42389.0,238734.0,15.93,36.0,H-SC,5.64,174.0,-0.24,1.74,27.91,XY24,NTT,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HAVELLS,2069.16,-5.19,-12.16,50.81,32.48,146394.0,-39871.0,288120.0,-16.61,36.0,X-MC,5.11,69.0,-0.27,2.10,0.00,X40,ATH,ELECTRICAL
13,BSOFT,831.70,-4.69,-26.41,104.90,50.79,108576.0,-37149.0,103504.0,-5.68,42.0,H-SC,8.77,171.0,-0.34,0.75,20.51,XR,ATH,IT
82,VOLTAS,1856.00,-3.92,5.11,38.14,45.20,76867.0,9798.0,201540.0,-5.08,36.0,X-MC,5.75,80.0,0.13,1.47,12.25,XY25,NTT,AC
0,5PAISA,593.00,-3.63,-30.47,62.38,12.90,94770.0,-66573.0,151923.0,123.56,54.0,H-SC,7.76,173.0,-0.70,1.11,26.32,OX40N,NTT,FINANCE
18,COFFEEDAY,80.00,-3.49,-43.93,141.25,35.27,89930.0,-49882.0,63667.0,-57.75,41.0,L-SC,5.42,268.0,-0.55,0.46,55.83,XR,NTT,HOTELS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.91,-6.06,121.32,107.91,168268.0,-8946.0,138698.0,-23.39,40.0,M-SC,9.91,220.0,-0.05,1.01,3.42,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.96,-19.14,23.69,0.02,33438.0,-33410.0,141149.0,115.40,40.0,M-SC,9.09,240.0,-1.00,1.03,22.78,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-2.43,-21.77,68.77,32.03,70924.0,-28703.0,103132.0,-15.37,49.0,H-SC,6.37,157.0,-0.40,0.75,15.52,OX40N,NTT,IT
77,TTKPRESTIG,770.00,2.25,-22.88,29.79,0.09,23152.0,-23061.0,77716.0,80.36,39.0,M-SC,4.69,253.0,-1.00,0.57,2.25,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.85,-23.76,48.84,13.47,100409.0,-64080.0,205587.0,-68.70,43.0,H-SC,4.22,159.0,-0.64,1.50,4.54,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,-2.46,3.17,15.56,19.23,24154.0,4774.0,155234.0,21.35,55.0,L-LC,10.41,182.0,0.2,1.13,25.12,XY25,ATH,MINING


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-2.50,-1.64,75.15,72.27,125246.0,-2787.0,166661.0,-15.61,32.0,M-LC,5.09,101.0,-0.02,1.22,4.43,XR,NTT,IT
25,FINCABLES,1641.55,-1.91,-6.06,121.32,107.91,168268.0,-8946.0,138698.0,-23.39,40.0,M-SC,9.91,220.0,-0.05,1.01,3.42,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.56,-7.18,122.70,106.72,140473.0,-8851.0,114485.0,-54.06,45.0,H-SC,8.60,138.0,-0.06,0.83,14.52,AR,ATH,MISC
37,IEX,219.00,-3.33,-10.12,65.36,48.63,117466.0,-20233.0,179721.0,-40.27,40.0,H-SC,13.17,137.0,-0.17,1.31,0.33,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-0.54,-7.59,46.77,35.63,90576.0,-15912.0,193662.0,-20.73,35.0,H-MC,4.49,119.0,-0.18,1.41,7.81,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.86,-31.87,130.73,57.19,290457.0,-103946.0,222181.0,-67.84,16.0,X-MC,12.34,58.0,-0.36,1.62,0.00,XY24,NTT,FMCG
43,ITC,452.00,-0.75,-18.95,36.68,10.78,71446.0,-45536.0,194782.0,-51.78,18.0,X-LC,11.74,1.0,-0.64,1.42,0.46,X40,NTT,FMCG
58,RELAXO,1176.00,-2.79,-52.60,221.31,52.31,152279.0,-76352.0,68808.0,-49.57,19.0,X-SC,8.89,92.0,-0.50,0.50,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.42,-42.53,135.25,35.20,100016.0,-54721.0,73949.0,-3.10,22.0,X-SC,17.89,93.0,-0.55,0.54,0.00,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-1.38,-10.12,49.31,34.19,94628.0,-21616.0,191904.0,-36.80,22.0,X-MC,5.12,60.0,-0.23,1.40,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.58,-1.09,-13.96,37.79,18.55,122978.0,-52811.0,325426.0,-25.89,46.0,X-LC,1.25,3.0,-0.43,2.37,10.71,X40,ATH,IT
20,DABUR,735.00,-0.67,-0.43,44.32,43.70,109475.0,-1058.0,247010.0,-8.69,49.0,X-MC,2.09,73.0,-0.01,1.80,14.56,XY24,BTT,FMCG
45,JIOFIN,387.00,-1.94,-13.06,43.17,24.48,100352.0,-34907.0,232458.0,-14.49,28.0,X-LC,2.10,37.0,-0.35,1.69,34.74,XY24,BTT,FINANCE
78,UNITDSPR,1644.00,-0.05,-2.73,24.15,20.77,55005.0,-6382.0,227762.0,-11.23,34.0,X-MC,2.22,70.0,-0.12,1.66,4.00,X40N,NTT,BREWERIES
33,HINDUNILVR,2922.00,0.58,-5.56,20.36,13.67,49430.0,-14283.0,242780.0,-12.61,62.0,X-LC,2.34,9.0,-0.29,1.77,14.61,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.86,-31.87,130.73,57.19,290457.0,-103946.0,222181.0,-67.84,16.0,X-MC,12.34,58.0,-0.36,1.62,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.42,-42.53,135.25,35.20,100016.0,-54721.0,73949.0,-3.10,22.0,X-SC,17.89,93.0,-0.55,0.54,0.0,X40,NTT,FOOTWEAR
9,BAJAJHLDNG,14451.52,-0.80,-5.55,36.81,29.22,66100.0,-10554.0,179571.0,-8.00,32.0,X-LC,12.23,29.0,-0.16,1.31,0.0,X40,ATH,FINANCE
8,BAJAJHFL,181.50,-1.19,-22.68,101.09,55.47,190152.0,-55184.0,188102.0,-31.47,25.0,X-MC,16.45,63.0,-0.29,1.37,0.0,X40N,ATH,FINANCE
31,HAVELLS,2069.16,-5.19,-12.16,50.81,32.48,146394.0,-39871.0,288120.0,-16.61,36.0,X-MC,5.11,69.0,-0.27,2.10,0.0,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-1.63,-31.99,110.03,42.84,48646.0,-20794.0,44212.0,-55.37,36.0,X-SC,23.35,83.0,-0.43,0.32,0.02,XY24,NTT,MISC
58,RELAXO,1176.00,-2.79,-52.60,221.31,52.31,152279.0,-76352.0,68808.0,-49.57,19.0,X-SC,8.89,92.0,-0.50,0.50,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.42,-42.53,135.25,35.20,100016.0,-54721.0,73949.0,-3.10,22.0,X-SC,17.89,93.0,-0.55,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-2.84,-9.60,37.25,24.08,42750.0,-12184.0,114766.0,-16.88,31.0,X-SC,6.41,91.0,-0.29,0.84,8.14,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.53,-16.74,72.15,43.32,97835.0,-27272.0,135600.0,-28.43,51.0,X-SC,8.11,90.0,-0.28,0.99,4.12,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.75,-18.95,36.68,10.78,71446.0,-45536.0,194782.0,-51.78,18.0,X-LC,11.74,1.0,-0.64,1.42,0.46,X40,NTT,FMCG
75,TMPV,600.00,-1.83,-30.79,77.67,22.98,121441.0,-69543.0,156355.0,-27.96,31.0,X-LC,3.62,2.0,-0.57,1.14,0.00,XY24,NTT,AUTO
73,TCS,4311.58,-1.09,-13.96,37.79,18.55,122978.0,-52811.0,325426.0,-25.89,46.0,X-LC,1.25,3.0,-0.43,2.37,10.71,X40,ATH,IT
81,VBL,671.64,-0.38,-0.19,35.93,35.67,113264.0,-606.0,315236.0,-12.02,53.0,X-LC,2.99,5.0,-0.01,2.30,13.64,X40N,ATH,FMCG
41,INFY,1972.00,-0.84,13.49,18.29,34.24,64946.0,42204.0,355092.0,-11.70,60.0,X-LC,7.41,6.0,0.65,2.59,22.89,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-3.25,-32.16,95.88,32.89,50222.0,-24826.0,52380.0,-747.22,59.0,L-MC,10.29,259.0,-0.49,0.38,44.25,XR,NTT,BANKS
17,COALINDIA,484.83,-2.46,3.17,15.56,19.23,24154.0,4774.0,155234.0,21.35,55.0,L-LC,10.41,182.0,0.20,1.13,25.12,XY25,ATH,MINING
36,ICICIPRULI,790.00,-0.67,8.64,20.34,30.73,43531.0,17014.0,214019.0,-16.79,47.0,X-MC,7.40,79.0,0.39,1.56,22.51,X40,ATH,INSURANCE
5,ASIANPAINT,3460.25,-1.23,-4.93,27.22,20.95,65154.0,-12408.0,239360.0,-5.83,41.0,X-LC,12.77,36.0,-0.19,1.75,28.95,X40,ATH,PAINTS
32,HCLTECH,1908.19,-1.49,9.76,12.82,23.83,34041.0,23610.0,265534.0,14.46,61.0,X-LC,6.36,13.0,0.69,1.94,27.40,X40,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-3.63,-30.47,62.38,12.90,94770.0,-66573.0,151923.0,123.56,54.0,H-SC,7.76,173.0,-0.70,1.11,26.32,OX40N,NTT,FINANCE
17,COALINDIA,484.83,-2.46,3.17,15.56,19.23,24154.0,4774.0,155234.0,21.35,55.0,L-LC,10.41,182.0,0.20,1.13,25.12,XY25,ATH,MINING
40,INDUSINDBK,1800.00,-3.25,-32.16,95.88,32.89,50222.0,-24826.0,52380.0,-747.22,59.0,L-MC,10.29,259.0,-0.49,0.38,44.25,XR,NTT,BANKS
62,SATIN,274.00,-0.77,-16.71,82.29,51.83,189370.0,-46159.0,230125.0,-25.10,60.0,H-SC,3.55,148.0,-0.24,1.68,12.10,XY24,NTT,FINANCE
33,HINDUNILVR,2922.00,0.58,-5.56,20.36,13.67,49430.0,-14283.0,242780.0,-12.61,62.0,X-LC,2.34,9.0,-0.29,1.77,14.61,XY25,NTT,FMCG


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.64
1,25,44.86
2,50,76.77


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.44
MC    30.68
LC    26.85
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.25
X40      24.19
X40N     12.89
XY25     10.95
XR        8.75
AR        8.36
OX40N     7.63
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.30
X-LC    23.45
H-SC    22.70
M-SC    11.08
X-SC     7.89
H-MC     4.54
M-MC     1.46
M-LC     1.22
L-LC     1.13
H-LC     1.05
L-SC     0.77
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.46,-10.03,46.12
IT,12.82,-21.61,82.36
FINANCE,11.08,-17.28,66.50
MISC,7.01,-34.96,90.17
PAINTS,5.81,-16.03,33.26
ELECTRICAL,5.79,-18.10,61.35
INSURANCE,4.74,-4.19,39.85
PHARMA,3.97,-7.83,41.77
BANKS,2.78,-40.08,124.96


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3370974.0,21
AR,1423326.0,10
XR,1290727.0,12
X40,1193450.0,15
X40N,973782.0,9
OX40N,744603.0,10
XY25,439827.0,7
SR,289812.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3679754.0,24
X-MC,1699431.0,16
M-SC,1538297.0,15
X-LC,997548.0,13
X-SC,837907.0,8
H-MC,456041.0,3
L-SC,184004.0,2
M-LC,125246.0,1
H-LC,88989.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1285103.0      6
           AR         952932.0      5
M-SC       XY24       877904.0      6
H-SC       XR         832105.0      7
X-MC       X40        569232.0      7
           XY24       465613.0      3
           X40N       429848.0      4
X-LC       X40        424665.0      6
M-SC       OX40N      330727.0      5
X-SC       X40N       328871.0      3
H-SC       OX40N      319802.0      4
X-SC       XY24       309483.0      3
H-SC       SR         289812.0      2
H-MC       AR         244963.0      2
X-MC       XY25       234738.0      2
X-LC       XY24       221793.0      2
           X40N       215063.0      2
H-MC       XY24       211078.0      1
X-SC       X40        199553.0      2
M-SC       XR         193224.0      2
           AR         136442.0      2
X-LC       XY25       136027.0      3
M-LC       XR         125246.0      1
L-SC       OX40N       94074.0      1
           XR          89930.0      1
H-LC       AR          88989.0      1
L-MC       XR          50222.0      1
M-MC       XY25        44908.0      1
L-LC       XY25        24154.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
